In [7]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from models import modelUtil as mu
from models import dummyModel as dm
from models import cnnModel as cm
from models import rnnModel as rm
from models import rnnModel2 as rm2
from utils import readin, yfReader, dataUtil as du, getPlots as gp
import EigenPortfolio as ep
import logging
import os

DATA_PATH = 'data/sp10/'
DATA_PATH_ALL = 'data/sp150'
D = 10
N = 10
c = 0.0001
epochs = 20
transCostParams = {
'c': np.array([ [c] for _ in range(D) ]),
'c0': c
}
baselineTransCostParams = np.zeros(D + 1) + c

resultsDirectory = 'results/allresults/' + du.getCurrentTimestamp() 
os.mkdir(resultsDirectory)

# now call logger.info to log
logger = du.setupLogger(resultsDirectory)

In [8]:
def main():
    print(du.getCurrentTimestamp())

    trainAndTestTrial()
    #rnnModelTrainingTrial()



In [11]:
def trainAndTestTrial():
    ## specify testing data
    # get data for testing
    dateSelected, stockPrice = du.getData(DATA_PATH, 'week')
    # get testing data index
    TestTimeIndex = range(10+(len(dateSelected)-10)/2+1, len(dateSelected))
    # get testing time period Date for estimating return (startDate, endDate)
    TestDate = [(dateSelected[i-1][0],dateSelected[i][0]) for i in TestTimeIndex]

    # get testing data index
    TestIndex = [i-N-1 for i in TestTimeIndex]
    # get training data index
    TrainIndex = range(TestIndex[0]) 

    returnTensor, prevReturnMatrix, nextReturnMatrix = du.getInputs(stockPrice, N)
    print(returnTensor.shape)
 
    ## get training data
    returnTensor_Train = np.array([returnTensor[_] for _ in TrainIndex])
    prevReturnMatrix_Train = np.array([prevReturnMatrix[_] for _ in TrainIndex])
    nextReturnMatrix_Train = np.array([nextReturnMatrix[_] for _ in TrainIndex])
    print(returnTensor_Train.shape)
    ## get testing data
    returnTensor_Test = np.array([returnTensor[_] for _ in TestIndex])
    prevReturnMatrix_Test = np.array([prevReturnMatrix[_] for _ in TestIndex])
    nextReturnMatrix_Test = np.array([nextReturnMatrix[_] for _ in TestIndex])    

    # generate xticks for plotting
    xticks = du.date2xtick(dateSelected[i] for i in TestTimeIndex)

    # epochs and tolerance for training 
    epochs = 500
    logger.info('total epochs: '+str(epochs))
    tol = 1e-7
    logger.info('tolerance: '+str(tol))
    
    # define model
    curModel = cm.CnnModel(D, N, transCostParams)
    L = 1

    model_info = curModel.get_model_info()
    logger.info('model basic config')
    logger.info(model_info)

    #quit()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ## train model
        preTotalGR = 0
        for e in range(epochs):
            logger.info('Beginning '+str(e)+'_th epoch')
            logger.info('')

            allActions, growthRates = mu.train1epoch(returnTensor_Train, prevReturnMatrix_Train, nextReturnMatrix_Train, curModel, sess)
            totalGR = du.prod(growthRates)

            logger.info('model total growth rate in training data: '+ str(totalGR))
            logger.info('')
            
            if np.abs(preTotalGR - totalGR) < tol:
                break
            else:
                preTotalGR = totalGR

        ## test model
        allActions, growthRates = mu.test1epoch(returnTensor_Test, prevReturnMatrix_Test, nextReturnMatrix_Test, curModel, sess)
        print(allActions[0])

        baselineGrowthRates = 1.0 + ep.baseline(stockPrice, TestTimeIndex, baselineTransCostParams)
        growthRates = growthRates[-len(baselineGrowthRates):]
        totalGR = du.prod(growthRates)
        baselineTotalGR = du.prod(baselineGrowthRates)
        logger.info('model total growth rate in testing data: '+ str(totalGR))
        logger.info('baseline total growth rate: '+str(baselineTotalGR))
        logger.info('')

if __name__=='__main__':
    main()

2017-12-03 09:29:48,786 INFO total epochs: 500
2017-12-03 09:29:48,786 INFO total epochs: 500
2017-12-03 09:29:48,789 INFO tolerance: 1e-07
2017-12-03 09:29:48,789 INFO tolerance: 1e-07


2017-12-03-09:29:48
(144, 10, 10)
(72, 10, 10)


2017-12-03 09:29:49,538 INFO model basic config
2017-12-03 09:29:49,538 INFO model basic config
2017-12-03 09:29:49,541 INFO None
2017-12-03 09:29:49,541 INFO None


model info
{"model_type": "CNNModel", "dropout": 0.5, "lr": 0.0001}



2017-12-03 09:29:50,974 INFO Beginning 0_th epoch
2017-12-03 09:29:50,974 INFO Beginning 0_th epoch
2017-12-03 09:29:50,980 INFO 
2017-12-03 09:29:50,980 INFO 
2017-12-03 09:29:51,269 INFO model total growth rate in training data: 1.09829472513
2017-12-03 09:29:51,269 INFO model total growth rate in training data: 1.09829472513
2017-12-03 09:29:51,272 INFO 
2017-12-03 09:29:51,272 INFO 
2017-12-03 09:29:51,276 INFO Beginning 1_th epoch
2017-12-03 09:29:51,276 INFO Beginning 1_th epoch
2017-12-03 09:29:51,279 INFO 
2017-12-03 09:29:51,279 INFO 
2017-12-03 09:29:51,481 INFO model total growth rate in training data: 1.09943832497
2017-12-03 09:29:51,481 INFO model total growth rate in training data: 1.09943832497
2017-12-03 09:29:51,487 INFO 
2017-12-03 09:29:51,487 INFO 
2017-12-03 09:29:51,491 INFO Beginning 2_th epoch
2017-12-03 09:29:51,491 INFO Beginning 2_th epoch
2017-12-03 09:29:51,499 INFO 
2017-12-03 09:29:51,499 INFO 
2017-12-03 09:29:51,715 INFO model total growth rate in trai

...

2017-12-03 09:30:42,429 INFO 
2017-12-03 09:30:42,429 INFO 
2017-12-03 09:30:42,432 INFO Beginning 315_th epoch
2017-12-03 09:30:42,432 INFO Beginning 315_th epoch
2017-12-03 09:30:42,435 INFO 
2017-12-03 09:30:42,435 INFO 
2017-12-03 09:30:42,586 INFO model total growth rate in training data: 1.87432166861
2017-12-03 09:30:42,586 INFO model total growth rate in training data: 1.87432166861
2017-12-03 09:30:42,589 INFO 
2017-12-03 09:30:42,589 INFO 
2017-12-03 09:30:42,592 INFO Beginning 316_th epoch
2017-12-03 09:30:42,592 INFO Beginning 316_th epoch
2017-12-03 09:30:42,595 INFO 
2017-12-03 09:30:42,595 INFO 
2017-12-03 09:30:42,750 INFO model total growth rate in training data: 1.87432307305
2017-12-03 09:30:42,750 INFO model total growth rate in training data: 1.87432307305
2017-12-03 09:30:42,753 INFO 
2017-12-03 09:30:42,753 INFO 
2017-12-03 09:30:42,757 INFO Beginning 317_th epoch
2017-12-03 09:30:42,757 INFO Beginning 317_th epoch
2017-12-03 09:30:42,760 INFO 
2017-12-03 09:30:4

2017-12-03 09:30:45,805 INFO 
2017-12-03 09:30:45,805 INFO 
2017-12-03 09:30:45,808 INFO Beginning 336_th epoch
2017-12-03 09:30:45,808 INFO Beginning 336_th epoch
2017-12-03 09:30:45,811 INFO 
2017-12-03 09:30:45,811 INFO 
2017-12-03 09:30:45,962 INFO model total growth rate in training data: 1.87434130557
2017-12-03 09:30:45,962 INFO model total growth rate in training data: 1.87434130557
2017-12-03 09:30:45,964 INFO 
2017-12-03 09:30:45,964 INFO 
2017-12-03 09:30:45,967 INFO Beginning 337_th epoch
2017-12-03 09:30:45,967 INFO Beginning 337_th epoch
2017-12-03 09:30:45,971 INFO 
2017-12-03 09:30:45,971 INFO 
2017-12-03 09:30:46,120 INFO model total growth rate in training data: 1.87434188075
2017-12-03 09:30:46,120 INFO model total growth rate in training data: 1.87434188075
2017-12-03 09:30:46,122 INFO 
2017-12-03 09:30:46,122 INFO 
2017-12-03 09:30:46,126 INFO Beginning 338_th epoch
2017-12-03 09:30:46,126 INFO Beginning 338_th epoch
2017-12-03 09:30:46,130 INFO 
2017-12-03 09:30:4

2017-12-03 09:30:49,192 INFO 
2017-12-03 09:30:49,192 INFO 
2017-12-03 09:30:49,195 INFO Beginning 357_th epoch
2017-12-03 09:30:49,195 INFO Beginning 357_th epoch
2017-12-03 09:30:49,198 INFO 
2017-12-03 09:30:49,198 INFO 
2017-12-03 09:30:49,349 INFO model total growth rate in training data: 1.87434824573
2017-12-03 09:30:49,349 INFO model total growth rate in training data: 1.87434824573
2017-12-03 09:30:49,352 INFO 
2017-12-03 09:30:49,352 INFO 
2017-12-03 09:30:49,355 INFO Beginning 358_th epoch
2017-12-03 09:30:49,355 INFO Beginning 358_th epoch
2017-12-03 09:30:49,358 INFO 
2017-12-03 09:30:49,358 INFO 
2017-12-03 09:30:49,511 INFO model total growth rate in training data: 1.87434842835
2017-12-03 09:30:49,511 INFO model total growth rate in training data: 1.87434842835
2017-12-03 09:30:49,514 INFO 
2017-12-03 09:30:49,514 INFO 
2017-12-03 09:30:49,517 INFO Beginning 359_th epoch
2017-12-03 09:30:49,517 INFO Beginning 359_th epoch
2017-12-03 09:30:49,521 INFO 
2017-12-03 09:30:4

[[  3.83622198e-13]
 [  2.19597014e-11]
 [  4.63934655e-11]
 [  1.57624226e-13]
 [  4.19577949e-11]
 [  6.21617146e-05]
 [  7.04459277e-14]
 [  9.99937773e-01]
 [  7.11138526e-08]
 [  6.27008482e-14]
 [  1.00000000e+00]]


2017-12-03 09:30:51,750 INFO model total growth rate in testing data: 1.77706143937
2017-12-03 09:30:51,750 INFO model total growth rate in testing data: 1.77706143937
2017-12-03 09:30:51,752 INFO baseline total growth rate: 1.52140300103
2017-12-03 09:30:51,752 INFO baseline total growth rate: 1.52140300103
2017-12-03 09:30:51,755 INFO 
2017-12-03 09:30:51,755 INFO 


In [31]:
print(1)

1
